# **Intitialize Spark & Import Libraries**

In [1]:
import requests
from pyspark.sql import SparkSession
import pandas as pd
# Initialize SparkSession
spark = SparkSession.builder.appName("CashFlowDataProcessing").getOrCreate()


# **Function to Fetch Data from Api and store them**

In [2]:
# Function to fetch, process, and save cash flow data for a given symbol
def fetch_cash_flow_data_and_process(symbol):
    url = "https://real-time-finance-data.p.rapidapi.com/company-cash-flow"
    querystring = {"symbol": f"{symbol}:NASDAQ", "period": "QUARTERLY", "language": "en"}
    headers = {
        "x-rapidapi-key": "976db0922amshc20f563c83bd3bcp1c9c36jsn8998095535a5",
        "x-rapidapi-host": "real-time-finance-data.p.rapidapi.com"
    }

    try:
        # Fetch data from API
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            data = response.json()  # Parse JSON response
            print(f"Data fetched successfully for {symbol}.")
        else:
            raise Exception(f"Failed to fetch data for {symbol}: {response.status_code} - {response.text}")

        # Extract the 'cash_flow' data
        cash_flows = data.get("data", {}).get("cash_flow", [])
        if not cash_flows:
            print(f"No cash flow data found for {symbol}.")
            return

        # Convert JSON data into an RDD and Spark DataFrame
        rdd = spark.sparkContext.parallelize(cash_flows)
        df = spark.read.json(rdd)
        print(f"Displaying the Spark DataFrame for {symbol}:")
        df.show(truncate=False)

        # Perform analysis: select specific columns
        print("Performing analysis on selected columns:")
        selected_df = df.select("date","cash_from_financing","cash_from_investing","cash_from_operations","net_income", "cash_from_operations", "free_cash_flow")
        selected_df.show()

        # Convert Spark DataFrame to Pandas DataFrame
        print("Converting Spark DataFrame to Pandas DataFrame...")
        pdf = selected_df.toPandas()

        # Convert 'date' column to datetime format
        pdf['date'] = pd.to_datetime(pdf['date'])

        # Save the Pandas DataFrame to a CSV file
        csv_path = f'{symbol}_financial_data.csv'
        pdf.to_csv(csv_path, index=False)
        print(f"Dataset for {symbol} saved as {csv_path}")

    except Exception as e:
        print(f"Error fetching or processing data for {symbol}: {e}")

# List of company symbols
symbols = ["AAPL", "MSFT", "NVDA", "AMZN", "TSLA"]

# Loop through each symbol and process the data
for symbol in symbols:
    print(f"Processing data for {symbol}...")
    fetch_cash_flow_data_and_process(symbol)


Processing data for AAPL...
Data fetched successfully for AAPL.
Displaying the Spark DataFrame for AAPL:
+-------------------+-------------------+--------------------+--------+----------+---+--------------+-----+------------------+-----------+----+
|cash_from_financing|cash_from_investing|cash_from_operations|currency|date      |day|free_cash_flow|month|net_change_in_cash|net_income |year|
+-------------------+-------------------+--------------------+--------+----------+---+--------------+-----+------------------+-----------+----+
|-24948000000       |1445000000         |26811000000         |USD     |2024-9-28 |28 |34538375000   |9    |3308000000        |14736000000|2024|
|-36017000000       |-127000000         |28858000000         |USD     |2024-6-29 |29 |22498000000   |6    |-7286000000       |21448000000|2024|
|-30433000000       |-310000000         |22690000000         |USD     |2024-3-30 |30 |20609500000   |3    |-8053000000       |23636000000|2024|
|-30585000000       |1927000000

# **copy and paste the code below in app.py after executing the streamlit app in collab**

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Cache the data for better performance
@st.cache_data
def load_data(file_path):
    return pd.read_csv(file_path)

# Sidebar for dataset selection
st.sidebar.title("Dataset Selection")
selected_dataset = st.sidebar.radio(
    "Choose a Dataset",
    [
        "Apple (AAPL)",
        "Amazon (AMZN)",
        "Microsoft (MSFT)",
        "NVIDIA (NVDA)",
        "Tesla (TSLA)"
    ]
)

# Map dataset choices to file paths (add your CSV paths)
dataset_mapping = {
    "Apple (AAPL)": "AAPL_financial_data.csv",
    "Amazon (AMZN)": "AMZN_financial_data.csv",
    "Microsoft (MSFT)": "MSFT_financial_data.csv",
    "NVIDIA (NVDA)": "NVDA_financial_data.csv",
    "Tesla (TSLA)": "TSLA_financial_data.csv",
}

# Load the selected dataset
df = load_data(dataset_mapping[selected_dataset])

# Convert 'date' column to datetime and sort the data by date
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# Streamlit App Title
st.title("Financial Data Dashboard")
st.write(f"### Selected Dataset: {selected_dataset}")

# Find the maximum and minimum Net Income values with their corresponding dates
max_net_income = df.loc[df['net_income'].idxmax()]
min_net_income = df.loc[df['net_income'].idxmin()]

# Find the maximum and minimum Cash From Operations values
max_cash_operations = df.loc[df['cash_from_operations'].idxmax()]
min_cash_operations = df.loc[df['cash_from_operations'].idxmin()]

# Find the maximum and minimum Free Cash Flow values
max_free_cash_flow = df.loc[df['free_cash_flow'].idxmax()]
min_free_cash_flow = df.loc[df['free_cash_flow'].idxmin()]

# Calculate max and min for Cash From Financing
max_cash_financing = df.loc[df['cash_from_financing'].idxmax()]
min_cash_financing = df.loc[df['cash_from_financing'].idxmin()]

# Calculate max and min for Cash From Investing
max_cash_investing = df.loc[df['cash_from_investing'].idxmax()]
min_cash_investing = df.loc[df['cash_from_investing'].idxmin()]

# Format large numbers for tooltips
def format_tooltip(value):
    if abs(value) >= 1e9:  # Billions
        return f"{value / 1e9:.2f}B"
    elif abs(value) >= 1e6:  # Millions
        return f"{value / 1e6:.2f}M"
    elif abs(value) >= 1e3:  # Thousands
        return f"{value / 1e3:.2f}K"
    return f"{value:.2f}"

# Add a formatted column for custom hover data
df['formatted_net_income'] = df['net_income'].apply(format_tooltip)

# Plot Net Income vs Date
fig_net_income = px.line(
    df,
    x='date',
    y='net_income',
    markers=True,
    title='Net Income Over Time',
    labels={"date": "Date", "net_income": "Net Income (USD)"},
    hover_data={'date': "|%Y-%m-%d", 'formatted_net_income': True},
)
fig_net_income.update_traces(
    hovertemplate="<b>Date:</b> %{x|%Y-%m-%d}<br>" +
                  "<b>Net Income:</b> %{customdata} USD<extra></extra>",
    customdata=df['formatted_net_income']
)

# Highlight Max and Min points for Net Income
fig_net_income.add_scatter(
    x=[max_net_income['date']],
    y=[max_net_income['net_income']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_net_income['net_income'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Net Income"
)

fig_net_income.add_scatter(
    x=[min_net_income['date']],
    y=[min_net_income['net_income']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_net_income['net_income'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Net Income"
)

# Plot Cash From Operations vs Date
fig_cash_operations = px.line(
    df,
    x='date',
    y='cash_from_operations',
    markers=True,
    title='Cash From Operations Over Time',
    labels={"date": "Date", "cash_from_operations": "Cash From Operations (USD)"}
)
fig_cash_operations.add_scatter(
    x=[max_cash_operations['date']],
    y=[max_cash_operations['cash_from_operations']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_cash_operations['cash_from_operations'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Cash From Operations"
)
fig_cash_operations.add_scatter(
    x=[min_cash_operations['date']],
    y=[min_cash_operations['cash_from_operations']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_cash_operations['cash_from_operations'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Cash From Operations"
)

# Plot Cash From Financing vs Date
fig_cash_financing = px.line(
    df,
    x='date',
    y='cash_from_financing',
    markers=True,
    title='Cash From Financing Over Time',
    labels={"date": "Date", "cash_from_financing": "Cash From Financing (USD)"}
)
fig_cash_financing.add_scatter(
    x=[max_cash_financing['date']],
    y=[max_cash_financing['cash_from_financing']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_cash_financing['cash_from_financing'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Cash From Financing"
)
fig_cash_financing.add_scatter(
    x=[min_cash_financing['date']],
    y=[min_cash_financing['cash_from_financing']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_cash_financing['cash_from_financing'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Cash From Financing"
)

# Plot Cash From Investing vs Date
fig_cash_investing = px.line(
    df,
    x='date',
    y='cash_from_investing',
    markers=True,
    title='Cash From Investing Over Time',
    labels={"date": "Date", "cash_from_investing": "Cash From Investing (USD)"}
)
fig_cash_investing.add_scatter(
    x=[max_cash_investing['date']],
    y=[max_cash_investing['cash_from_investing']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_cash_investing['cash_from_investing'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Cash From Investing"
)
fig_cash_investing.add_scatter(
    x=[min_cash_investing['date']],
    y=[min_cash_investing['cash_from_investing']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_cash_investing['cash_from_investing'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Cash From Investing"
)


# Plot Free Cash Flow vs Date
fig_free_cash_flow = px.line(
    df,
    x='date',
    y='free_cash_flow',
    markers=True,
    title='Free Cash Flow Over Time',
    labels={"date": "Date", "free_cash_flow": "Free Cash Flow (USD)"}
)
fig_free_cash_flow.add_scatter(
    x=[max_free_cash_flow['date']],
    y=[max_free_cash_flow['free_cash_flow']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_free_cash_flow['free_cash_flow'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Free Cash Flow"
)
fig_free_cash_flow.add_scatter(
    x=[min_free_cash_flow['date']],
    y=[min_free_cash_flow['free_cash_flow']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_free_cash_flow['free_cash_flow'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Free Cash Flow"
)

# NET INCOME VS TIME

# Display the Net Income Plot
st.plotly_chart(fig_net_income, use_container_width=True)

# Define company-specific Net Income thresholds (Max and Min)
net_income_thresholds = {
    "Apple (AAPL)": {"max": 35e9, "moderate": 25e9, "min": 20e9},
    "Amazon (AMZN)": {"max": 30e9, "moderate": 20e9, "min": 15e9},
    "Microsoft (MSFT)": {"max": 28e9, "moderate": 18e9, "min": 14e9},
    "NVIDIA (NVDA)": {"max": 20e9, "moderate": 15e9, "min": 10e9},
    "Tesla (TSLA)": {"max": 15e9, "moderate": 10e9, "min": 7e9},
}

# Get the thresholds for the selected company
company_net_income_thresholds = net_income_thresholds[selected_dataset]

# Dynamic comments for Net Income graph based on min and max points
st.write("### Investor Insights: Net Income")

# Insights for Max Net Income
if max_net_income['net_income'] > company_net_income_thresholds["max"]:
    st.success(f"📈 **Exceptional Profitability**: Net Income reached a peak of {format_tooltip(max_net_income['net_income'])} in {max_net_income['date'].strftime('%b %Y')}. This reflects strong business performance and robust operational efficiency.")
elif max_net_income['net_income'] > company_net_income_thresholds["moderate"]:
    st.warning(f"⚠️ **Moderate Performance**: Net Income peaked at {format_tooltip(max_net_income['net_income'])} in {max_net_income['date'].strftime('%b %Y')}. Evaluate growth opportunities to strengthen profitability.")
else:
    st.error(f"🔻 **Below Expectations**: Net Income peaked at only {format_tooltip(max_net_income['net_income'])}. Investigate challenges affecting profitability.")

# Insights for Min Net Income
if min_net_income['net_income'] < company_net_income_thresholds["min"]:
    st.error(f"📉 **Profitability Concern**: Net Income dropped to a low of {format_tooltip(min_net_income['net_income'])} in {min_net_income['date'].strftime('%b %Y')}. This may indicate operational inefficiencies or market-related challenges.")
else:
    st.info(f"ℹ️ **Stable Performance**: The minimum Net Income remained at {format_tooltip(min_net_income['net_income'])}, suggesting steady profitability.")

# General Investor Takeaways
st.info("""
💡 **Investor Tip**:
- **Consistent Net Income growth** is a sign of strong profitability and operational excellence.
- **Sudden dips** may indicate challenges in cost management, revenue streams, or external factors.
- Evaluate trends over time to identify long-term sustainability and compare performance with peers in the same industry.
""")


# CFO VS TIME

# Display the interactive plots in Streamlit
st.plotly_chart(fig_cash_operations, use_container_width=True)

# Define company-specific CFO thresholds (Max and Min)
cfo_thresholds = {
    "Apple (AAPL)": {"max": 40e9, "moderate": 30e9, "min": 25e9},
    "Amazon (AMZN)": {"max": 35e9, "moderate": 25e9, "min": 20e9},
    "Microsoft (MSFT)": {"max": 30e9, "moderate": 22e9, "min": 18e9},
    "NVIDIA (NVDA)": {"max": 20e9, "moderate": 15e9, "min": 10e9},
    "Tesla (TSLA)": {"max": 15e9, "moderate": 10e9, "min": 7e9},
}

# Get the thresholds for the selected company
company_thresholds = cfo_thresholds[selected_dataset]

# Dynamic comments for Cash From Operations graph based on min and max points
st.write("### Investor Insights: Cash From Operations")

# Insights for Max Cash From Operations
if max_cash_operations['cash_from_operations'] > company_thresholds["max"]:
    st.success(f"📈 **Exceptional Performance**: Cash From Operations reached a peak of {format_tooltip(max_cash_operations['cash_from_operations'])} in {max_cash_operations['date'].strftime('%b %Y')}. This reflects superior operational efficiency and cash generation.")
elif max_cash_operations['cash_from_operations'] > company_thresholds["moderate"]:
    st.warning(f"⚠️ **Moderate Performance**: Cash From Operations peaked at {format_tooltip(max_cash_operations['cash_from_operations'])} in {max_cash_operations['date'].strftime('%b %Y')}. Look for opportunities to enhance growth.")
else:
    st.error(f"🔻 **Below Expectations**: Cash From Operations peaked at only {format_tooltip(max_cash_operations['cash_from_operations'])}. Investors should investigate potential challenges in cash generation.")

# Insights for Min Cash From Operations
if min_cash_operations['cash_from_operations'] < company_thresholds["min"]:
    st.error(f"📉 **Significant Concern**: Cash From Operations dropped to a low of {format_tooltip(min_cash_operations['cash_from_operations'])} in {min_cash_operations['date'].strftime('%b %Y')}. This indicates potential inefficiencies or operational struggles.")
else:
    st.info(f"ℹ️ **Stable Operations**: The minimum Cash From Operations remained steady at {format_tooltip(min_cash_operations['cash_from_operations'])}, indicating consistency.")

# General Investor Takeaways
st.info("""
💡 **Investor Tip**:
- Consistently high **Cash From Operations** reflects a strong and sustainable business model.
- Sudden drops in CFO warrant further investigation into operational issues.
- Compare CFO trends to competitors in the same sector to evaluate performance effectively.
""")


# Cash Financing vs time

# Add a unique key for the chart
st.plotly_chart(fig_cash_financing, use_container_width=True, key="cash_financing_chart_1")

# Define company-specific Cash From Financing thresholds
cash_financing_thresholds = {
    "Apple (AAPL)": {"max": 30e9, "moderate": 20e9, "min": -10e9},
    "Amazon (AMZN)": {"max": 25e9, "moderate": 15e9, "min": -12e9},
    "Microsoft (MSFT)": {"max": 20e9, "moderate": 10e9, "min": -8e9},
    "NVIDIA (NVDA)": {"max": 15e9, "moderate": 8e9, "min": -6e9},
    "Tesla (TSLA)": {"max": 12e9, "moderate": 6e9, "min": -5e9},
}

# Get thresholds for the selected company
company_cash_financing_thresholds = cash_financing_thresholds[selected_dataset]

# Display the Cash From Financing graph
# st.write("## Cash From Financing Over Time")
fig_cash_financing = px.line(
    df,
    x='date',
    y='cash_from_financing',
    markers=True,
    title="Cash From Financing Over Time",
    labels={"date": "Date", "cash_from_financing": "Cash From Financing (USD)"},
)

# Highlight Max and Min Cash From Financing
fig_cash_financing.add_scatter(
    x=[max_cash_financing['date']],
    y=[max_cash_financing['cash_from_financing']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_cash_financing['cash_from_financing'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Cash From Financing"
)

fig_cash_financing.add_scatter(
    x=[min_cash_financing['date']],
    y=[min_cash_financing['cash_from_financing']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_cash_financing['cash_from_financing'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Cash From Financing"
)

# Dynamic Investor Insights for Cash From Financing
st.write("### Investor Insights: Cash From Financing")

# Insights based on Cash From Financing max and min thresholds
if max_cash_financing['cash_from_financing'] > company_cash_financing_thresholds['max']:
    st.success(f"📈 **Strong Financing Activity**: Cash From Financing peaked at {format_tooltip(max_cash_financing['cash_from_financing'])} in {max_cash_financing['date'].strftime('%b %Y')}. This suggests effective capital-raising strategies for growth or expansion.")
elif max_cash_financing['cash_from_financing'] > company_cash_financing_thresholds['moderate']:
    st.warning(f"⚠️ **Moderate Financing Activity**: Cash From Financing reached {format_tooltip(max_cash_financing['cash_from_financing'])}. Monitor for signs of leverage and capital usage.")
else:
    st.info(f"ℹ️ **Low Financing Activity**: Cash From Financing peaked at {format_tooltip(max_cash_financing['cash_from_financing'])}, indicating conservative financial management.")

if min_cash_financing['cash_from_financing'] < company_cash_financing_thresholds['min']:
    st.error(f"📉 **Debt Repayment Focus**: Cash From Financing dropped to {format_tooltip(min_cash_financing['cash_from_financing'])} in {min_cash_financing['date'].strftime('%b %Y')}. This suggests significant repayments or reduced external funding.")
else:
    st.info(f"ℹ️ **Stable Financing**: The minimum Cash From Financing remained at {format_tooltip(min_cash_financing['cash_from_financing'])}, reflecting steady financial positioning.")

# General Investor Tip
st.info("""
💡 **Investor Tip**:
- **Positive Cash From Financing** indicates fundraising through debt or equity issuance. Analyze if the raised capital is used effectively for growth initiatives.
- **Negative Cash From Financing** reflects debt repayments or share buybacks, which can signal strong financial health if sustainable.
- Investors should evaluate how the company balances external financing with operational cash flow.
""")


# Define company-specific Cash From Investing thresholds
cash_investing_thresholds = {
    "Apple (AAPL)": {"max": 25e9, "moderate": 15e9, "min": -20e9},
    "Amazon (AMZN)": {"max": 20e9, "moderate": 12e9, "min": -18e9},
    "Microsoft (MSFT)": {"max": 18e9, "moderate": 10e9, "min": -15e9},
    "NVIDIA (NVDA)": {"max": 12e9, "moderate": 8e9, "min": -10e9},
    "Tesla (TSLA)": {"max": 10e9, "moderate": 6e9, "min": -8e9},
}

# Get thresholds for the selected company
company_cash_investing_thresholds = cash_investing_thresholds[selected_dataset]

# Display the Cash From Investing graph
# st.write("## Cash From Investing Over Time")
fig_cash_investing = px.line(
    df,
    x='date',
    y='cash_from_investing',
    markers=True,
    title="Cash From Investing Over Time",
    labels={"date": "Date", "cash_from_investing": "Cash From Investing (USD)"},
)

# Highlight Max and Min Cash From Investing
fig_cash_investing.add_scatter(
    x=[max_cash_investing['date']],
    y=[max_cash_investing['cash_from_investing']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_cash_investing['cash_from_investing'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Cash From Investing"
)

fig_cash_investing.add_scatter(
    x=[min_cash_investing['date']],
    y=[min_cash_investing['cash_from_investing']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_cash_investing['cash_from_investing'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Cash From Investing"
)

# Add a unique key for the chart
st.plotly_chart(fig_cash_investing, use_container_width=True, key="cash_investing_chart_1")

# Dynamic Investor Insights for Cash From Investing
st.write("### Investor Insights: Cash From Investing")

# Insights based on Cash From Investing max and min thresholds
if max_cash_investing['cash_from_investing'] > company_cash_investing_thresholds['max']:
    st.success(f"📈 **Aggressive Investment Strategy**: Cash From Investing peaked at {format_tooltip(max_cash_investing['cash_from_investing'])} in {max_cash_investing['date'].strftime('%b %Y')}. This suggests the company is investing heavily in growth opportunities.")
elif max_cash_investing['cash_from_investing'] > company_cash_investing_thresholds['moderate']:
    st.warning(f"⚠️ **Moderate Investment Activity**: Cash From Investing reached {format_tooltip(max_cash_investing['cash_from_investing'])}. Evaluate ROI on these investments to ensure sustainable growth.")
else:
    st.info(f"ℹ️ **Low Investment Activity**: Cash From Investing peaked at {format_tooltip(max_cash_investing['cash_from_investing'])}, indicating a conservative investment strategy.")

if min_cash_investing['cash_from_investing'] < company_cash_investing_thresholds['min']:
    st.error(f"📉 **Asset Sales or Divestments**: Cash From Investing dropped to {format_tooltip(min_cash_investing['cash_from_investing'])} in {min_cash_investing['date'].strftime('%b %Y')}. This could indicate asset sales or reduced capital expenditures.")
else:
    st.info(f"ℹ️ **Steady Investing**: The minimum Cash From Investing remained at {format_tooltip(min_cash_investing['cash_from_investing'])}, suggesting consistent capital management.")

# General Investor Tip
st.info("""
💡 **Investor Tip**:
- **Positive Cash From Investing** may indicate asset sales but can be unsustainable over time.
- **Negative Cash From Investing** suggests capital expenditures, acquisitions, or R&D, which are good signs if they lead to future growth.
- Investors should evaluate the return on investment (ROI) for these expenditures to assess the company's long-term strategy.
""")


# Define company-specific Free Cash Flow thresholds
free_cash_flow_thresholds = {
    "Apple (AAPL)": {"max": 35e9, "moderate": 25e9, "min": 15e9},
    "Amazon (AMZN)": {"max": 30e9, "moderate": 20e9, "min": 10e9},
    "Microsoft (MSFT)": {"max": 28e9, "moderate": 18e9, "min": 8e9},
    "NVIDIA (NVDA)": {"max": 20e9, "moderate": 15e9, "min": 7e9},
    "Tesla (TSLA)": {"max": 15e9, "moderate": 10e9, "min": 5e9},
}

# Get thresholds for the selected company
company_fcf_thresholds = free_cash_flow_thresholds[selected_dataset]

# Display the Free Cash Flow graph
# st.write("## Free Cash Flow Over Time")
fig_free_cash_flow = px.line(
    df,
    x='date',
    y='free_cash_flow',
    markers=True,
    title="Free Cash Flow Over Time",
    labels={"date": "Date", "free_cash_flow": "Free Cash Flow (USD)"},
)

# Highlight Max and Min Free Cash Flow
fig_free_cash_flow.add_scatter(
    x=[max_free_cash_flow['date']],
    y=[max_free_cash_flow['free_cash_flow']],
    mode='markers+text',
    text=[f"Max: {format_tooltip(max_free_cash_flow['free_cash_flow'])}"],
    textposition="top center",
    marker=dict(size=10, color="green"),
    name="Max Free Cash Flow"
)

fig_free_cash_flow.add_scatter(
    x=[min_free_cash_flow['date']],
    y=[min_free_cash_flow['free_cash_flow']],
    mode='markers+text',
    text=[f"Min: {format_tooltip(min_free_cash_flow['free_cash_flow'])}"],
    textposition="bottom center",
    marker=dict(size=10, color="red"),
    name="Min Free Cash Flow"
)

# Add a unique key for the chart
st.plotly_chart(fig_free_cash_flow, use_container_width=True, key="free_cash_flow_chart_1")

# Dynamic Investor Insights for Free Cash Flow
st.write("### Investor Insights: Free Cash Flow")

# Insights based on Free Cash Flow max and min thresholds
if max_free_cash_flow['free_cash_flow'] > company_fcf_thresholds['max']:
    st.success(f"📈 **Exceptional Free Cash Flow**: FCF peaked at {format_tooltip(max_free_cash_flow['free_cash_flow'])} in {max_free_cash_flow['date'].strftime('%b %Y')}. This suggests strong liquidity and the ability to fund dividends, investments, or debt repayment.")
elif max_free_cash_flow['free_cash_flow'] > company_fcf_thresholds['moderate']:
    st.warning(f"⚠️ **Moderate Free Cash Flow**: FCF reached {format_tooltip(max_free_cash_flow['free_cash_flow'])}. Monitor for improvements to sustain higher cash availability.")
else:
    st.error(f"🔻 **Below Expectations**: Free Cash Flow peaked at only {format_tooltip(max_free_cash_flow['free_cash_flow'])}. This could indicate challenges in cash generation or high capital expenditures.")

if min_free_cash_flow['free_cash_flow'] < company_fcf_thresholds['min']:
    st.error(f"📉 **Liquidity Concern**: Free Cash Flow dropped to {format_tooltip(min_free_cash_flow['free_cash_flow'])} in {min_free_cash_flow['date'].strftime('%b %Y')}. Evaluate high costs or cash outflows affecting liquidity.")
else:
    st.info(f"ℹ️ **Steady Cash Generation**: Minimum Free Cash Flow remained at {format_tooltip(min_free_cash_flow['free_cash_flow'])}, indicating consistent cash management.")

# General Investor Tip
st.info("""
💡 **Investor Tip**:
- **Free Cash Flow (FCF)** reflects the cash available after covering capital expenditures. Positive FCF is critical for funding dividends, reducing debt, or reinvesting in growth.
- **High FCF** signals financial strength, while negative FCF may indicate excessive spending or operational challenges.
- Analyze trends over time to ensure consistent cash generation and evaluate its sustainability.
""")


# Function to format large numbers (Millions, Billions)
def format_large_numbers(value):
    if abs(value) >= 1e9:  # Billions
        return f"{value / 1e9:.2f}B"
    elif abs(value) >= 1e6:  # Millions
        return f"{value / 1e6:.2f}M"
    return f"{value:.2f}"

# Aggregate data by year
df['year'] = df['date'].dt.year  # Extract the year from the date column
aggregated_data = df.groupby('year').agg({
    'cash_from_operations': 'sum',
    'cash_from_financing': 'sum',
    'cash_from_investing': 'sum'
}).reset_index()

# Melt the data for a single bar plot
aggregated_melted = pd.melt(
    aggregated_data,
    id_vars=['year'],  # Year is kept as the identifier
    value_vars=['cash_from_operations', 'cash_from_financing', 'cash_from_investing'],
    var_name='Metric',  # Column for metric names
    value_name='Amount'  # Column for corresponding values
)

# Apply formatting to the "Amount" column for display
aggregated_melted['formatted_amount'] = aggregated_melted['Amount'].apply(format_large_numbers)

# Create a single bar chart
fig_aggregated_bar = px.bar(
    aggregated_melted,
    x='year',
    y='Amount',
    color='Metric',
    barmode='group',  # Grouped bars (or use 'stack' for stacked bars)
    title="Aggregated Cash Flow Metrics by Year",
    labels={"year": "Year", "Amount": "Amount (USD)"}
)

# Customize hovertemplate without 'Metric' and with formatted numbers
fig_aggregated_bar.update_traces(
    hovertemplate="<b>Year:</b> %{x}<br>" +
                  "<b>Amount:</b> %{customdata} USD<extra></extra>",
    customdata=aggregated_melted['formatted_amount']
)

# Display the bar chart in Streamlit
# st.write("## Aggregated Cash Flow Metrics by Year")
st.plotly_chart(fig_aggregated_bar, use_container_width=True)

# Dynamic Investor Insights for Aggregated Cash Flow Metrics
st.write("### Investor Insights: Aggregated Cash Flow Metrics")

# Get total values for each metric across all years
total_operations = aggregated_data['cash_from_operations'].sum()
total_financing = aggregated_data['cash_from_financing'].sum()
total_investing = aggregated_data['cash_from_investing'].sum()

# Analyze Cash From Operations
if total_operations > 50e9:
    st.success(f"📈 **Strong Operational Cash Flow**: Total Cash From Operations across years is {format_large_numbers(total_operations)}. This indicates strong core business performance and cash generation.")
elif total_operations > 20e9:
    st.warning(f"⚠️ **Moderate Operations**: Total Cash From Operations is {format_large_numbers(total_operations)}. Evaluate growth strategies to enhance operational efficiency.")
else:
    st.error(f"🔻 **Weak Operations**: Total Cash From Operations is {format_large_numbers(total_operations)}, suggesting challenges in generating cash from core operations.")

# Analyze Cash From Financing
if total_financing > 10e9:
    st.success(f"💰 **Capital Infusion**: Total Cash From Financing is {format_large_numbers(total_financing)}, indicating successful fundraising through equity or debt to fund growth.")
elif total_financing < -10e9:
    st.error(f"📉 **Debt Repayment Focus**: Total Cash From Financing is {format_large_numbers(total_financing)}, reflecting significant repayments or reduced external financing.")
else:
    st.info(f"ℹ️ **Stable Financing**: Total Cash From Financing is {format_large_numbers(total_financing)}, indicating balanced financial activities.")

# Analyze Cash From Investing
if total_investing < -40e9:
    st.success(f"🏗️ **Aggressive Investments**: Total Cash From Investing is {format_large_numbers(total_investing)}. The company is investing heavily in assets, acquisitions, or R&D for future growth.")
elif total_investing < -20e9:
    st.warning(f"⚠️ **Moderate Investments**: Total Cash From Investing is {format_large_numbers(total_investing)}. Monitor ROI to ensure effective capital allocation.")
else:
    st.info(f"ℹ️ **Conservative Investing**: Total Cash From Investing is {format_large_numbers(total_investing)}, indicating cautious capital management.")

# General Investor Tips
st.info("""
💡 **Investor Tip**:
- **Cash From Operations**: Consistently high values reflect strong core business performance.
- **Cash From Financing**: Positive values indicate capital raises; negative values show debt repayments or buybacks.
- **Cash From Investing**: Heavy investments may signal growth strategies but require monitoring ROI.
- Compare trends across years to evaluate financial stability and growth sustainability.
""")

# Calculate correlation between key metrics
st.write("### Correlation Between Financial Metrics")
correlation = df[['net_income', 'free_cash_flow', 'cash_from_operations','cash_from_financing','cash_from_investing']].corr()
st.write(correlation)

# Plot Correlation Matrix using Plotly
fig_corr = go.Figure(
    data=go.Heatmap(
        z=correlation.values,
        x=correlation.columns,
        y=correlation.index,
        colorscale=[[0, 'blue'], [0.5, 'white'], [1, 'red']],
        hoverongaps=False,
        colorbar=dict(title="Correlation"),
    )
)
fig_corr.update_layout(
    title="Correlation Matrix Between Financial Metrics",
    xaxis=dict(title="Metrics"),
    yaxis=dict(title="Metrics"),
    margin=dict(l=40, r=40, t=40, b=40),
)

# Display Plotly heatmap in Streamlit
st.plotly_chart(fig_corr, use_container_width=True)

# Plot highly correlated metrics without repetition
st.write("### Highly Correlated Metrics")
high_corr_pairs = correlation.where((correlation > 0.7) & (correlation < 1)).stack().reset_index()
high_corr_pairs.columns = ['Metric 1', 'Metric 2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Metric 1'] < high_corr_pairs['Metric 2']]

if not high_corr_pairs.empty:
    for _, row in high_corr_pairs.iterrows():
        metric1 = row['Metric 1']
        metric2 = row['Metric 2']
        st.write(f"**Plotting {metric1} vs {metric2}** (Correlation: {row['Correlation']:.2f})")
        scatter_fig = px.scatter(
            df,
            x=metric1,
            y=metric2,
            title=f"{metric1} vs {metric2}",
            labels={metric1: metric1, metric2: metric2},
            hover_name='date',
            trendline="ols"
        )
        st.plotly_chart(scatter_fig, use_container_width=True)
else:
    st.write("No highly correlated metrics found.")

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Prepare data for the predictive model
st.write("## Predictive Model: Future Trend of Net Income")

# Extract the year and month as numerical features for modeling
df['year_month'] = df['date'].dt.year + (df['date'].dt.month - 1) / 12  # Convert dates to numerical format
X = df[['year_month']].values  # Feature: Numerical year/month
y = df['net_income'].values    # Target: Net Income

# Scale data for better model performance
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_scaled, y)

from sklearn.metrics import r2_score

# Replace the deprecated 'M' with 'ME' in predictive model date generation
future_years = pd.date_range(df['date'].max(), periods=12, freq='ME')  # 'ME' stands for month end
future_year_months = future_years.year + (future_years.month - 1) / 12
future_X = scaler.transform(future_year_months.values.reshape(-1, 1))

# Generate predictions
future_predictions = model.predict(future_X)

# Calculate model accuracy (R² score)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

# Combine historical and future predictions into a DataFrame
future_df = pd.DataFrame({
    'date': future_years,
    'net_income': future_predictions
})

# Visualization of Historical and Future Trend
fig_predictive = go.Figure()

# Add Historical Net Income
fig_predictive.add_trace(go.Scatter(
    x=df['date'],
    y=df['net_income'],
    mode='lines+markers',
    name='Historical Net Income',
    line=dict(color='blue'),
    hovertemplate="<b>Date:</b> %{x|%Y-%m-%d}<br>" +
                  "<b>Net Income:</b> %{y:,.2f} USD<extra></extra>"
))

# Add Predicted Future Trend
fig_predictive.add_trace(go.Scatter(
    x=future_df['date'],
    y=future_df['net_income'],
    mode='lines+markers',
    name='Predicted Net Income',
    line=dict(color='red', dash='dash'),
    hovertemplate="<b>Date:</b> %{x|%Y-%m-%d}<br>" +
                  "<b>Predicted Net Income:</b> %{y:,.2f} USD<extra></extra>"
))

# Update layout
fig_predictive.update_layout(
    title="Net Income: Historical and Predicted Trend",
    xaxis_title="Date",
    yaxis_title="Net Income (USD)",
    legend=dict(x=0, y=1, bgcolor='rgba(255,255,255,0.5)'),
    hovermode="x unified"
)

# Display the predictive model visualization
st.plotly_chart(fig_predictive, use_container_width=True)

# Inform the user about the method and display model accuracy
st.write(f"""
The predictive trend uses a **Linear Regression Model** trained on historical net income data.
This provides a simple forecast for future net income over the next 12 months.

📊 **Model Accuracy**: R² Score = {r2:.2f}
""")

# Dynamic Investor Insights for Predictive Model
st.write("### Investor Insights: Predictive Model for Net Income")

# Formatting function for readability
def format_number_readable(value):
    if abs(value) >= 1e9:
        return f"{value / 1e9:.2f}B"
    elif abs(value) >= 1e6:
        return f"{value / 1e6:.2f}M"
    return f"{value:.2f}"

# Analyze Historical and Predicted Trends
historical_max = df['net_income'].max()
predicted_max = future_df['net_income'].max()
historical_min = df['net_income'].min()

# Insights for Historical Net Income
if historical_max > 30e9:
    st.success(f"📈 **Strong Historical Performance**: Historical Net Income peaked at {format_number_readable(historical_max)} USD, indicating robust profitability in the past.")
else:
    st.warning(f"⚠️ **Moderate Historical Performance**: The peak Net Income was {format_number_readable(historical_max)} USD. There may be room for improvement in profitability.")

if historical_min < 15e9:
    st.error(f"📉 **Downtrend Concern**: Historical Net Income dropped as low as {format_number_readable(historical_min)} USD. Evaluate past challenges in profitability or market pressures.")

# Insights for Predicted Net Income
if predicted_max > historical_max:
    st.success(f"📊 **Promising Future**: The model predicts Net Income could rise to {format_number_readable(predicted_max)} USD, surpassing historical highs. This suggests potential growth opportunities.")
elif predicted_max > 20e9:
    st.info(f"ℹ️ **Stable Future Trend**: Predicted Net Income is expected to reach {format_number_readable(predicted_max)} USD, indicating a stable recovery or moderate growth.")
else:
    st.warning(f"⚠️ **Cautious Outlook**: Predicted Net Income is forecasted to peak at {format_number_readable(predicted_max)} USD, which may signal slower growth. Monitor operational strategies to drive profitability.")

# Compare Historical and Predicted Trends
st.write("""
💡 **Investor Tip**:
- Compare **historical performance** with the predicted trend to evaluate the company's growth trajectory.
- A rising predicted trend suggests opportunities for expansion, while a flat or declining prediction warrants caution.
- Investors should consider external factors (market trends, competition) that could influence the company's ability to meet or exceed these predictions.
""")


# **SETTING UP THE STREAMLIT APP**

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [4]:
%%writefile app.py

Writing app.py


In [5]:
!pip install streamlit pyngrok --quiet

In [6]:
! wget -q -O - ipv4.icanhazip.com

34.16.149.116


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.149.116:8501

⠴⠦your url is: https://khaki-hats-shout.loca.lt
2024-12-18 07:38:03.735 Session with id 4aa6f916-7681-4688-92e1-d0175749ca73 is already connected! Connecting to a new session.


# **Stop the spark session**

In [ ]:
spark.stop()
